<a href="https://colab.research.google.com/github/Rahul-Gadadhar/Diabetes-Advisor/blob/main/Diabetes_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AI-Powered Personalized Diabetes Nutrition Advisor using RNN


---




In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score
import google.generativeai as genai
from google.colab import userdata
from IPython.display import Markdown, display

###Loading the Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

diabetes_data = pd.read_csv(r'/content/drive/My Drive/Datasets/diabetes_patient_data.csv')
pred_food_data = pd.read_csv(r'/content/drive/My Drive/Datasets/pred_food.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Preprocessing the Diabetes Data

In [ ]:
label_encoders = {}

diabetes_data.columns = diabetes_data.columns.str.strip()
pred_food_data.columns = pred_food_data.columns.str.strip()

for column in ['Gender', 'Type of Diabetes', 'Current Medications']:
    le = LabelEncoder()
    diabetes_data[column] = le.fit_transform(diabetes_data[column])
    label_encoders[column] = le

for column in ['Suitable for Diabetes', 'Suitable for Blood Pressure']:
    le = LabelEncoder()
    pred_food_data[column] = le.fit_transform(pred_food_data[column])
    label_encoders[column] = le

###Combining and Scaling the data

In [ ]:
combined_data = pd.DataFrame({
    'Age': diabetes_data['Age'],
    'Gender': diabetes_data['Gender'],
    'Type of Diabetes': diabetes_data['Type of Diabetes'],
    'Blood Sugar Levels (mg/dL)': diabetes_data['Blood Sugar Levels (mg/dL)'],
    'Glycemic Index': pred_food_data['Glycemic Index'].iloc[:len(diabetes_data)],
    'Calories': pred_food_data['Calories'].iloc[:len(diabetes_data)],
    'Carbohydrates': pred_food_data['Carbohydrates'].iloc[:len(diabetes_data)],
    'Protein': pred_food_data['Protein'].iloc[:len(diabetes_data)],
    'Fat': pred_food_data['Fat'].iloc[:len(diabetes_data)],
    'Sodium Content': pred_food_data['Sodium Content'].iloc[:len(diabetes_data)],
    'Potassium Content': pred_food_data['Potassium Content'].iloc[:len(diabetes_data)],
    'Magnesium Content': pred_food_data['Magnesium Content'].iloc[:len(diabetes_data)],
    'Calcium Content': pred_food_data['Calcium Content'].iloc[:len(diabetes_data)],
    'Fiber Content': pred_food_data['Fiber Content'].iloc[:len(diabetes_data)]
})

scaler = MinMaxScaler()

features_to_use = ['Age', 'Blood Sugar Levels (mg/dL)', 'Glycemic Index', 'Calories',
                   'Carbohydrates', 'Protein', 'Fat', 'Sodium Content',
                   'Potassium Content', 'Magnesium Content', 'Calcium Content',
                   'Fiber Content']

combined_data[features_to_use] = scaler.fit_transform(
    combined_data[features_to_use]
)

###Defining X and y variables

In [ ]:
X = combined_data[features_to_use].values

y = np.random.randint(0, len(pred_food_data), size=(len(combined_data),))
y = np.eye(len(pred_food_data))[y]

X = X.reshape((X.shape[0], 1, X.shape[1]))

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

###Defining the RNN Model

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

input_size = X.shape[2]
hidden_size = 50
output_size = len(pred_food_data)

model = RNNModel(input_size, hidden_size, output_size)


###Defining loss function and optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

###Training the RNN

In [ ]:
epochs = 500
batch_size = 10

for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_tensor.size(0))
    epoch_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    all_true_labels = []
    all_predicted_labels = []

    for i in range(0, X_tensor.size(0), batch_size):
        indices = permutation[i:i + batch_size]
        batch_X, batch_y = X_tensor[indices], y_tensor[indices]

        outputs = model(batch_X)
        loss = criterion(outputs, torch.argmax(batch_y, dim=1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        true_labels = torch.argmax(batch_y, dim=1)
        all_true_labels.extend(true_labels.cpu().numpy())

        all_predicted_labels.extend(predicted.cpu().numpy())
        total_samples += batch_y.size(0)
        correct_predictions += (predicted == torch.argmax(batch_y, dim=1)).sum().item()

    epoch_loss /= len(X_tensor) / batch_size
    epoch_accuracy = (correct_predictions / total_samples) * 100

    precision = precision_score(all_true_labels, all_predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(all_true_labels, all_predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(all_true_labels, all_predicted_labels, average='weighted', zero_division=0)

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")

#Save the model
torch.save(model.state_dict(), 'diabetes_prediction_model.pth')

Epoch 1/500, Loss: 6.7703, Accuracy: 0.00%
Epoch 2/500, Loss: 6.6848, Accuracy: 0.00%
Epoch 3/500, Loss: 6.5810, Accuracy: 0.00%
Epoch 4/500, Loss: 6.4453, Accuracy: 0.99%
Epoch 5/500, Loss: 6.3000, Accuracy: 0.00%
Epoch 6/500, Loss: 6.0917, Accuracy: 0.00%
Epoch 7/500, Loss: 5.8487, Accuracy: 0.99%
Epoch 8/500, Loss: 5.6518, Accuracy: 0.00%
Epoch 9/500, Loss: 5.4601, Accuracy: 0.99%
Epoch 10/500, Loss: 5.3067, Accuracy: 3.96%
Epoch 11/500, Loss: 5.2128, Accuracy: 2.97%
Epoch 12/500, Loss: 5.0826, Accuracy: 3.96%
Epoch 13/500, Loss: 5.0855, Accuracy: 3.96%
Epoch 14/500, Loss: 5.0445, Accuracy: 4.95%
Epoch 15/500, Loss: 5.0164, Accuracy: 2.97%
Epoch 16/500, Loss: 5.0021, Accuracy: 2.97%
Epoch 17/500, Loss: 4.9765, Accuracy: 2.97%
Epoch 18/500, Loss: 4.9576, Accuracy: 2.97%
Epoch 19/500, Loss: 4.9181, Accuracy: 2.97%
Epoch 20/500, Loss: 4.9487, Accuracy: 3.96%
Epoch 21/500, Loss: 4.9034, Accuracy: 1.98%
Epoch 22/500, Loss: 4.9025, Accuracy: 2.97%
Epoch 23/500, Loss: 4.8976, Accuracy: 1.9

###Preprocessing User Input

In [ ]:
def preprocess_user_input(age, gender, diabetes_type, blood_sugar_level):
    gender_encoded = label_encoders['Gender'].transform([gender])[0]
    diabetes_type_encoded = label_encoders['Type of Diabetes'].transform([diabetes_type])[0]

    # Create a DataFrame with column names
    new_user_data = pd.DataFrame([{
        'Age': age,
        'Blood Sugar Levels (mg/dL)': blood_sugar_level,
        'Glycemic Index': 0,
        'Calories': 0,
        'Carbohydrates': 0,
        'Protein': 0,
        'Fat': 0,
        'Sodium Content': 0,
        'Potassium Content': 0,
        'Magnesium Content': 0,
        'Calcium Content': 0,
        'Fiber Content': 0
    }])

    # Scale the new user data
    new_user_data_scaled = scaler.transform(new_user_data)
    return torch.tensor(new_user_data_scaled.reshape((1, 1, len(new_user_data_scaled[0]))), dtype=torch.float32)

def recommend_foods(user_input, blood_sugar_level,suggestions):
    model.eval()
    with torch.no_grad():
        predicted_food_index = model(user_input)
        recommended_indices = torch.argsort(predicted_food_index[0], descending=True)[:suggestions]

    recommended_foods = pred_food_data.iloc[recommended_indices.numpy()]

    # Main Recommendations Table
    print("\nRecommended Foods:")
    print("=" * 90)
    print(f"{'Food Name':<20}{'Glycemic Index':<17}{'Blood Sugar Change':<20}{'Impact':<20}{'Calories':<10}")
    print("=" * 90)
    for index, row in recommended_foods.iterrows():
        gi = row['Glycemic Index']
        expected_blood_sugar_change = blood_sugar_level + (gi / 100 * blood_sugar_level)

        if expected_blood_sugar_change < 140:
            category = 'Low Impact'
        elif 140 <= expected_blood_sugar_change <= 180:
            category = 'Moderate Impact'
        else:
            category = 'High Impact'

        print(f"{row['Food Name']:<20}{gi:<17}{expected_blood_sugar_change:<20}{category:<20}{row['Calories']:<10}")
        print("-" * 90)

    # Detailed Nutritional Table
    print("\nNutritional Details for Recommended Foods:")
    print("=" * 120)
    print(f"{'Food Name':<20}{'Carbs(g)':<12}{'Protein(g)':<12}{'Fat(g)':<10}{'Sodium(mg)':<12}"
          f"{'Potassium(mg)':<15}{'Magnesium(mg)':<15}{'Calcium(mg)':<12}{'Fiber (g)':<10}")
    print("=" * 120)
    for index, row in recommended_foods.iterrows():
        print(f"{row['Food Name']:<20}{row['Carbohydrates']:<12}{row['Protein']:<12}{row['Fat']:<10}"
              f"{row['Sodium Content']:<12}{row['Potassium Content']:<15}{row['Magnesium Content']:<15}"
              f"{row['Calcium Content']:<12}{row['Fiber Content']:<10}")
        print("-" * 120)

    return recommended_foods

###Generating AI response

In [ ]:
def explain_with_gemini(recommended_foods):

    recommendations_summary = []
    for _, row in recommended_foods.iterrows():
        recommendations_summary.append({
            'Food Name': row['Food Name'],
            'Glycemic Index': row['Glycemic Index'],
            'Carbs': row['Carbohydrates'],
            'Protein': row['Protein'],
            'Fat': row['Fat'],
            'Sodium': row['Sodium Content'],
            'Potassium': row['Potassium Content'],
            'Magnesium': row['Magnesium Content'],
            'Calcium': row['Calcium Content'],
            'Fiber': row['Fiber Content']
        })

    recommendations_text = "\n".join([
        f"Food Name: {rec['Food Name']}, Glycemic Index: {rec['Glycemic Index']}, Carbs: {rec['Carbs']}g, Protein: {rec['Protein']}g, Fat: {rec['Fat']}g, Sodium: {rec['Sodium']}mg, Potassium: {rec['Potassium']}mg, Magnesium: {rec['Magnesium']}mg, Calcium: {rec['Calcium']}mg, Fiber: {rec['Fiber']}g."
        for rec in recommendations_summary
    ])

    # Generate response with Gemini
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)

    model = genai.GenerativeModel("gemini-2.0-flash-exp")
    response = model.generate_content(
        f"Provide a well structured explanation of how the following foods can help with diabetic control based on their nutritional content:\n{recommendations_text}. Make sure its short and concise."
    )

    # Display Gemini's response
    print("\nGemini's Explanation:")
    print("=" * 120)
    display(Markdown(response.text))
    print("-" * 120)


###User Input and Displaying results

In [ ]:
age = int(input("What's your Age: "))
gender = input("Enter your Gender (Male/Female): ").strip().capitalize()
diabetes_type = input("Enter your type of Diabetes (Type 1/Type 2): ").strip().capitalize()
suggestions = int(input("How many food suggestions would you like?: "))

while True:
    blood_sugar_level = float(input("Enter your blood sugar level (range 70-180 mg/dL): "))
    if 70 <= blood_sugar_level <= 180:
        break
    else:
        print("Please enter a blood sugar level within the typical range (70-180 mg/dL).")

user_input = preprocess_user_input(age, gender, diabetes_type, blood_sugar_level)
recommended_foods = recommend_foods(user_input, blood_sugar_level,suggestions)
print(f"\nAccuracy Metrics:")
print('-'*50)
print(f"Epochs: {epochs} ; Loss: {epoch_loss:.4f} ; Accuracy: {epoch_accuracy:.2f}%")
print(f"Precision: {(precision*100):.2f}% \nRecall: {(recall*100):.2f}% \nF1-Score: {(f1*100):.2f}%")
print('-'*50)
explain_with_gemini(recommended_foods)

What's your Age: 21
Enter your Gender (Male/Female): Male
Enter your type of Diabetes (Type 1/Type 2): Type 1
How many food suggestions would you like?: 5
Enter your blood sugar level (range 70-180 mg/dL): 123

Recommended Foods:
Food Name           Glycemic Index   Blood Sugar Change  Impact              Calories  
Masoor Dal          25               153.75              Moderate Impact     352       
------------------------------------------------------------------------------------------
Greek Yogurt        11               136.53              Low Impact          59        
------------------------------------------------------------------------------------------
Pineapple           59               195.57              High Impact         50        
------------------------------------------------------------------------------------------
Asparagus           15               141.45              Moderate Impact     20        
---------------------------------------------------------

Okay, here's a breakdown of how each food can contribute to diabetes management based on their nutritional profiles:

**Understanding the Key Factors for Diabetes Control:**

*   **Glycemic Index (GI):**  Measures how quickly a food raises blood sugar levels. Lower GI is better for stable blood sugar.
*   **Fiber:** Slows down sugar absorption, improving blood sugar control and promoting satiety.
*   **Protein:** Helps with satiety, slows down glucose absorption, and aids in muscle health, important for overall metabolic health.
*   **Carbohydrates:** Need to be managed; focusing on complex carbs and balancing intake is important.
*   **Potassium:** Important for blood pressure control, often affected by diabetes.
*   **Magnesium:** Plays a role in insulin sensitivity and blood sugar regulation.

**Food Analysis and Diabetes Benefits:**

1.  **Masoor Dal (Lentils):**
    *   **Low GI (25):**  Leads to a slow, gradual rise in blood sugar.
    *   **High Fiber (7.0g):** Further slows sugar absorption, improves blood sugar control and satiety.
    *   **High Protein (24.0g):** Contributes to satiety and muscle maintenance.
    *   **High Potassium (1202mg):** Beneficial for blood pressure management.
    *   **Good Magnesium (36mg):** Helps with insulin sensitivity.
    *   **Overall:**  Excellent for managing blood sugar due to its combination of low GI, high fiber, and high protein.

2.  **Greek Yogurt:**
    *   **Very Low GI (11):** Minimal impact on blood sugar.
    *   **Good Protein (10.0g):** Promotes satiety and aids in stable blood sugar levels.
    *   **Good Calcium (119mg):** Important for overall health and bone density.
    *   **Overall:** Great for a low-carb, high-protein option with minimal impact on blood glucose.

3.  **Pineapple:**
    *   **Moderate GI (59):**  Has a moderate impact on blood sugar. Portion control is crucial.
    *   **Moderate Carb (13.0g):** Must be eaten in moderation due to sugar content.
    *   **Moderate Fiber (1.4g):** Limited benefit to slowing sugar absorption.
    *   **Overall:**  Should be consumed in moderation due to its higher GI. Opt for fresh fruit over processed forms like juice or canned.

4.  **Asparagus:**
    *   **Very Low GI (15):**  Minimal impact on blood sugar levels.
    *   **Good Fiber (2.1g):** Helps with blood sugar control and satiety.
    *   **Good Protein (2.2g):**  Provides a small boost to satiety.
     *   **Good Potassium (202mg):** Beneficial for blood pressure management.
    *   **Overall:** An excellent, low-carb vegetable choice with minimal impact on blood sugar.

5.  **Carrots:**
    *   **Low GI (39):**  Leads to a slow rise in blood sugar.
    *   **Good Fiber (2.8g):** Aids in controlling blood sugar levels and satiety.
    *   **Good Potassium (320mg):**  Beneficial for blood pressure management.
    *   **Overall:**  A good, low-GI vegetable option, beneficial as a part of a balanced diabetic diet.

**Key Takeaways for Diabetes Control:**

*   Prioritize foods with **low GI** to minimize blood sugar spikes.
*   Include **high-fiber** foods to slow down glucose absorption.
*   Incorporate **high-protein** options for satiety and muscle health.
*   Be mindful of portion sizes, especially with moderate-GI foods like pineapple.
*   Include nutrient-rich options containing minerals like potassium and magnesium.
*   **Balance** all these nutrients within your overall diet.

*Disclaimer: This information is for general knowledge purposes only and does not constitute medical advice. It's important to consult a healthcare professional or registered dietitian for personalized dietary recommendations related to diabetes management.*


------------------------------------------------------------------------------------------------------------------------
